## Data preprocessing codes

In [221]:
%matplotlib inline

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import missingno as msno
import utils
from sklearn.impute import KNNImputer

In [222]:
#loading all dataframes 
df = utils.load('data/fma_metadata/tracks.csv')
genres_df = utils.load('data/fma_metadata/genres.csv')
echonest_df = utils.load('data/fma_metadata/echonest.csv')

#### DF1 df_global: raw dataset (outliers not cleaned) with all tracks without echonest data (100'000 + rows)

In [223]:
df_global = df.copy()

In [224]:
df_global.artist.members

track_id
2         Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...
3         Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...
5         Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...
10                                 Kurt Vile, the Violators
20                                             Nicky Cook\n
                                ...                        
155316                           GILLIAN/JENNA/DECLAN/JAIME
155317                           GILLIAN/JENNA/DECLAN/JAIME
155318                           GILLIAN/JENNA/DECLAN/JAIME
155319                           GILLIAN/JENNA/DECLAN/JAIME
155320    Alishia Taiping (lead vocals, bass) \nDan Pier...
Name: members, Length: 106574, dtype: object

In [225]:
# Data cleaning
#data cleaning CODE
df_global.drop(df_global.loc[(df_global.album.comments == -1),:].index, inplace = True)
df_global.drop(df_global.loc[df_global.artist.comments==-1].index, inplace=True)

#for rows where date_released > date_created: replace values with values from date_created
df_global.loc[df_global.album.date_released > df_global.album.date_created, ('album', 'date_released')] = df_global.loc[df_global.album.date_released > df_global.album.date_created, ('album', 'date_created')]

#producer and engineer to binary
df_global.loc[df_global.album.engineer.notna(),('album','engineer')] =  1
df_global.loc[:,('album','engineer')].fillna(0,inplace = True)

df_global.loc[df_global.album.producer.notna(),('album','producer')] =  1
df_global.loc[:,('album','producer')].fillna(0,inplace = True)


In [226]:
#drop entire columns
df_global.drop(columns = [('album','information')], inplace = True)
df_global.drop(columns = [('album','tags')], inplace = True)
df_global.drop(columns= [('album','date_created')], inplace =  True)

df_global.drop(columns = [('artist','active_year_begin')], inplace = True)
df_global.drop(columns = [('artist','active_year_end')], inplace = True)
df_global.drop(columns = [('artist','associated_labels')], inplace = True)
df_global.drop(columns = [('artist','bio')], inplace = True)
df_global.drop(columns = [('artist','date_created')], inplace = True)
df_global.drop(columns = [('artist','location')], inplace = True)
df_global.drop(columns = [('artist','latitude')], inplace = True)
df_global.drop(columns = [('artist','longitude')], inplace = True)
df_global.drop(columns = [('artist','name')], inplace = True)
df_global.drop(columns = [('artist','website')], inplace = True)
df_global.drop(columns = [('artist','wikipedia_page')], inplace = True)
df_global.drop(columns = [('artist','related_projects')], inplace = True)

df_global.drop(columns = [('track','composer')], inplace = True)
df_global.drop(columns = [('track','date_created')], inplace = True)
df_global.drop(columns = [('track','lyricist')], inplace = True)
df_global.drop(columns = [('track','tags')], inplace = True)
df_global.drop(columns = [('track','information')], inplace = True)
df_global.drop(columns = [('track','publisher')], inplace = True)
df_global.drop(columns = [('track','number')], inplace = True)
df_global.drop(columns = [('track','title')], inplace = True)
df_global.drop(columns = [('track','license')], inplace = True)
df_global.drop(columns = [('track','language_code')], inplace = True)

In [244]:
#artisti member
df_global.loc[:, ('artist','members')] = np.where(df_global.artist.members.isna(), 1, 0)

#use track.date_recorded to impute album.date_released. Then drop track.date_recorded
df_global.loc[(df_global.track.date_recorded.notna()) & (df_global.album.date_released.isna()), ('album','date_released')] = df_global.loc[(df_global.track.date_recorded.notna()) & (df_global.album.date_released.isna()), ('track','date_recorded')]
df_global.drop(columns = [('track','date_recorded')], inplace = True)

#genres
#1 -> drop rows with no genres at all
df_global.drop(df_global.loc[df_global.track.genres_all.str.len() == 0].index, inplace = True)
#2 -> clean genre top  all: abbiamo creato una nuova colonna uguale a genres_all dalla quale abbiamo rimosso tutti gli id che non sono top level and ADD NEW COLUMN CREATED TO df_global
genres_top_list = list(set(genres_df.top_level.to_list()))
genre_top_all = pd.DataFrame(df_global.track.genres_all)
for i, lista in genre_top_all.iterrows():
    temp_list = []
    for item in lista[0]:
        if item not in genres_top_list:
            temp_list.append(item)
    for j in temp_list:
        lista[0].remove(j)
df_global.track = df_global.track.join(genre_top_all.rename(columns={'genres_all' : 'genres_top_all'}),how = 'outer')
#3 -> fill genre_top empty with "Mixed" (because they have mixed genres)
df_global.loc[:,('track','genre_top')].fillna('Mixed',inplace = True)

#building global_track df_global
global_track = df_global.copy()

global_track.rename(columns ={'artist_members':'artist_solo'}, inplace = True)

AttributeError: 'DataFrame' object has no attribute 'date_recorded'

In [228]:
global_track.columns = global_track.columns.map('_'.join).str.strip('_')

In [229]:
#date released
global_track.loc[:,'album_date_released'] = global_track.groupby('album_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_date_released'] = global_track.groupby('artist_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_date_released'].fillna(echonest_df.echonest.metadata.album_date, inplace = True)
#covert album_date_released to years only
global_track.loc[:,'album_date_released'] = pd.to_datetime(global_track.loc[:,'album_date_released']).dt.year

In [230]:
global_track.loc[:,'album_type'] = global_track.groupby('album_id')['album_type'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)

In [231]:
#REPLACING MISSING VALUES BY IDS
#global_track.loc[:,'album_type'] = global_track.groupby('album_id')['album_type'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)

#album_date_released

#groupby
global_track.loc[:,'album_date_released'] = global_track.groupby('album_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_date_released'] = global_track.groupby('artist_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
global_track.loc[:,'album_date_released'].fillna(echonest_df.echonest.metadata.album_date, inplace = True)

In [232]:
#KNN imputer
imputer = KNNImputer(n_neighbors=2) #define the k nearest neighbors   
df_imputed = imputer.fit_transform(global_track[['album_date_released', 'artist_members','album_tracks','track_listens']])
global_track.loc[:,'album_date_released'].fillna(pd.DataFrame(df_imputed)[0].isna().sum(), inplace = True)

In [233]:
global_track.to_csv('global_track.csv', index = True)

#### DF2 echo_track: clean dataframe (tracks with echonest info, 10'000 + rows)

In [234]:
df_echo = df.copy()

In [235]:
# Data cleaning
#data cleaning CODE
df_echo.drop(df_echo.loc[(df_echo.album.comments == -1),:].index, inplace = True)
df_echo.drop(df_echo.loc[df_echo.artist.comments==-1].index, inplace=True)

#for rows where date_released > date_created: replace values with values from date_created
df_echo.loc[df_echo.album.date_released > df_echo.album.date_created, ('album', 'date_released')] = df_echo.loc[df_echo.album.date_released > df_echo.album.date_created, ('album', 'date_created')]
#drop('album','date_created')

#producer and engineer to binary
df_echo.loc[df_echo.album.engineer.notna(),('album','engineer')] =  1
df_echo.loc[:,('album','engineer')].fillna(0,inplace = True)
df_echo.loc[df_echo.album.producer.notna(),('album','producer')] =  1
df_echo.loc[:,('album','producer')].fillna(0,inplace = True)

In [236]:
#drop entire columns
df_echo.drop(columns = [('album','information')], inplace = True)
df_echo.drop(columns = [('album','tags')], inplace = True)
df_echo.drop(columns= [('album','date_created')], inplace =  True)

df_echo.drop(columns = [('artist','active_year_begin')], inplace = True)
df_echo.drop(columns = [('artist','active_year_end')], inplace = True)
df_echo.drop(columns = [('artist','associated_labels')], inplace = True)
df_echo.drop(columns = [('artist','bio')], inplace = True)
df_echo.drop(columns = [('artist','date_created')], inplace = True)
df_echo.drop(columns = [('artist','location')], inplace = True)
df_echo.drop(columns = [('artist','latitude')], inplace = True)
df_echo.drop(columns = [('artist','longitude')], inplace = True)
df_echo.drop(columns = [('artist','name')], inplace = True)
df_echo.drop(columns = [('artist','website')], inplace = True)
df_echo.drop(columns = [('artist','wikipedia_page')], inplace = True)
df_echo.drop(columns = [('artist','related_projects')], inplace = True)

df_echo.drop(columns = [('track','composer')], inplace = True)
df_echo.drop(columns = [('track','date_created')], inplace = True)
df_echo.drop(columns = [('track','lyricist')], inplace = True)
df_echo.drop(columns = [('track','tags')], inplace = True)
df_echo.drop(columns = [('track','information')], inplace = True)
df_echo.drop(columns = [('track','publisher')], inplace = True)
df_echo.drop(columns = [('track','number')], inplace = True)
df_echo.drop(columns = [('track','title')], inplace = True)
df_echo.drop(columns = [('track','license')], inplace = True)
df_echo.drop(columns = [('track','language_code')], inplace = True)

In [237]:
#artisti member
df_echo.loc[:, ('artist','members')] = np.where(df_echo.artist.members.isna(), 1, 0)

#use track.date_recorded to impute album.date_released. Then drop track.date_recorded
df_echo.loc[(df_echo.track.date_recorded.notna()) & (df_echo.album.date_released.isna()), ('album','date_released')] = df_echo.loc[(df_echo.track.date_recorded.notna()) & (df_echo.album.date_released.isna()), ('track','date_recorded')]
df_echo.drop(columns = [('track','date_recorded')], inplace = True)

#bit_rate
df_echo.loc[:,('track','bit_rate')] = df_echo.track.bit_rate/1000
df_echo.loc[df_echo.track.bit_rate < 0, ('track', 'bit_rate')] = 320

In [238]:
#genres
#1 -> drop rows with no genres at all
df_echo.drop(df_echo.loc[df_echo.track.genres_all.str.len() == 0].index, inplace = True)
#2 -> clean genre top  all: abbiamo creato una nuova colonna uguale a genres_all dalla quale abbiamo rimosso tutti gli id che non sono top level and ADD NEW COLUMN CREATED TO df_echo
genres_top_list = list(set(genres_df.top_level.to_list()))
genre_top_all = pd.DataFrame(df_echo.track.genres_all)
for i, lista in genre_top_all.iterrows():
    temp_list = []
    for item in lista[0]:
        if item not in genres_top_list:
            temp_list.append(item)
    for j in temp_list:
        lista[0].remove(j)
df_echo.track = df_echo.track.join(genre_top_all.rename(columns={'genres_all' : 'genres_top_all'}),how = 'outer')
df_echo.track = df_echo.track.map(ast.literal_eval)
#3 -> fill genre_top empty with "Mixed" (because they have mixed genres)
df_echo.loc[:,('track','genre_top')].fillna('Mixed',inplace = True)

In [239]:
#building echo_track dataframe
df_echo.columns = df_echo.columns.map('_'.join).str.strip('_')
echo_to_merge = echonest_df.iloc[:,:8].copy()
echo_to_merge.columns = echonest_df.iloc[:,:8].columns.droplevel(0).map('_'.join).str.strip('_')
echo_track = pd.merge(echo_to_merge, df_echo, on ='track_id')

echo_track.rename(columns ={'artist_members':'artist_solo'}, inplace = True)

In [240]:
#covert album_date_released to years only
echo_track.loc[:,'album_date_released'] = echo_track.loc[:,'album_date_released'].dt.year

In [241]:
#REPLEACING MISSING VALUES BY IDS
echo_track.loc[:,'album_type'] = echo_track.groupby('album_id')['album_type'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)
echo_track.loc[:,'album_date_released'] = echo_track.groupby('album_id')['album_date_released'].apply(lambda x: x.fillna(x.value_counts().index[0]) if x.value_counts().size != 0 else x)

#KNN imputer
imputer_echo = KNNImputer(n_neighbors=2) #define the k nearest neighbors   
df_echo_imputed = imputer_echo.fit_transform(echo_track.iloc[:,6:10])
echo_track.loc[:,'album_date_released'].fillna(pd.DataFrame(df_echo_imputed)[0].isna().sum(), inplace = True)


In [242]:
echo_track.to_csv('echo_track.csv', index = True)

In [243]:
#end